# Visualizing

We often want to visualize one or more exoplanet populations in fairly standard ways. Here we summarize some predefined visualizations for populations and explain how you can create your own multi-panel, multi-population visualizations with `exoatlas`.

## Built-in Visualizations 

`exoatlas` has a few pre-defined visualizations. The quickest way to dive into visualizing your exoplanet population might be to start with one of these. Below, we show you how to build up your own visualizations from scratch.

In [ ]:
import exoatlas as ea
ea.version()

In [ ]:
exoplanets = ea.Exoplanets()
solar = ea.SolarSystem()

In [ ]:
ea.physical_summary([exoplanets, solar]);

In [ ]:
ea.observable_summary([exoplanets, solar]);

In [ ]:
## Making Your Own Visualizations

🥺 Ha! Sorry! Nothing's been written here yet! 🥺

+ Include the default mega-plots of lots of panels.
+ Explain how to use individual panels (`MassRadius`).
+ Explain how to define your own panels.
+ Include easy wrapper/tutorial for making your own multi-panel plots.
+ Fix thumbtack plots, and show how to make movies.
